In [ ]:
import sys 
sys.path.append(rf"/nfs/home/genovese/thesis-wildfire-genovese/src")
from importlib import reload
import utils
reload(utils)
from utils import *
data_folder = "/nfs/home/genovese/thesis-wildfire-genovese/data/"

In [ ]:
# fixed per year
agricensus = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/gathering_geojson/per_comune/agricensus/2020.geojson')

In [ ]:
def loader(a, b, c, d, e):
    istruzione = gpd.read_file(f'/nfs/home/genovese/thesis-wildfire-genovese/data/gathering_geojson/per_comune/ISTRUZIONE/{a}.geojson')
    occupazione = gpd.read_file(f'/nfs/home/genovese/thesis-wildfire-genovese/data/gathering_geojson/per_comune/OCCUPAZIONE/{b}.geojson')
    ifc= gpd.read_file(f'/nfs/home/genovese/thesis-wildfire-genovese/data/gathering_geojson/per_comune/IFC/{c}.geojson')
    popolazione = gpd.read_file(f'/nfs/home/genovese/thesis-wildfire-genovese/data/gathering_geojson/per_comune/popolazione/{d}.geojson')
    redditi = gpd.read_file(f'/nfs/home/genovese/thesis-wildfire-genovese/data/gathering_geojson/per_comune/redditi/{e}.geojson')

    return istruzione, occupazione, ifc, popolazione, redditi

In [ ]:
i18, o18, ifc18, p16, r16 = loader(2018, 2018, 2018, 2016, 2016)

In [ ]:
tot16 = i18.merge(o18, how='outer', on='geometry'
                  ).merge(ifc18, how='outer', on='geometry'
                          ).merge(p16, how='outer', on='geometry'
                                  ).merge(r16, how='outer', on='geometry')

In [ ]:
import missingno as msno
msno.bar(tot16)

In [ ]:
def find_neighbors(gdf, row):
    neighbors = []
    for idx, row2 in gdf.iterrows():
        if (row2['geometry'].touches(row['geometry'])):
            neighbors.append(idx)
    return neighbors


def fill_missing_value(gdf, row, var):
    if pd.isna(row[var]):  # If it is missing...
        neighbors = find_neighbors(gdf, row)
        if neighbors:
            neighbor_val = gdf.loc[neighbors, var]
            # Calculate the average of neighbors (ignoring NaN values)
            if str(gdf[var].dtype) in ['float64']:
                return neighbor_val.mean()
            else:
                m = neighbor_val.mode()
                if m.shape[0] >= 1:
                    return m[0]
                else:
                    print('ERROR: ', m.shape)
    return row[var]

In [ ]:
# tot16.drop(columns=['65_studente/ssa'], inplace=True)

In [ ]:
def fix(row, var):
    if type(row[var]) == pd.core.series.Series:
        return pd.Series(row[var]).iloc[0][0]
    else:
        return row[var]

In [ ]:
for col in tot16.columns:
    tot16.loc[:,col] = tot16[[col]].apply(lambda x: fix(x, col), axis=1)

In [ ]:
for col in tqdm(tot16.columns[tot16.isna().any()]):
    tot16.loc[:,col] = tot16[[col, 'geometry']].apply(lambda x: fill_missing_value(tot16, x, col), axis=1)

In [ ]:
save_clean_data(tot16, 'per_comune_2016_provvisorio', '/nfs/home/genovese/thesis-wildfire-genovese/data/gathering_geojson/per_comune')

In [ ]:
msno.bar(tot16)

In [ ]:
msno.heatmap(tot16)

---